In [ ]:
import os
import torch
from typing import Literal
from datetime import datetime

# which_infra:Literal["onyxia", "datalab_gcp", "local"] = os.environ["WHICH_INFRA"]
which_infra:Literal["onyxia", "datalab_gcp", "local"] = "local"

device: torch.device = torch.device("cpu") # default device to cpu
date = datetime.now().strftime("%m_%d_%Y-%Hh_%Mmin")
match which_infra:
    case "local":
        device = torch.device("mps")
        output_dir = f"../bucket/results_{date}"
    case "datalab_gcp":
        device = torch.device("cuda")
        output_dir = f"../../bucket/results_{date}"
    case "onyxia": 
        device = torch.device("cuda")
        output_dir = f"../../bucket/results_{date}" # todo: look how to access onyxia s3 buckets
    case _:
        raise ValueError(f"Unexpected value for environment variable WHICH_INFRA. Accepted values are : 'onyxia', 'datalab_gcp' and 'local'.")


In [ ]:
import torch

# to change when on datalab
datalab: bool = (which_infra == "datalab_gcp") # whether the notebook is running on datalab gcp (True) or locally on mac with mps (False)
resume_from_checkpoint: bool = False # resume from last cp. If True; set checkpoint path to an existing checkpoint
data_prop = 1
do_lora:bool = True # whether to do lora fine tuning or juste last layer fine tuning
date = "24_04_25"


model_name: str = "meta-llama/Llama-3.2-1B-Instruct"
torch_dtype: torch.dtype = torch.bfloat16
checkpoint_path: str = "../bucket/results/checkpoint-" if datalab else "/Users/mgg/dev/projets/fine-tuning/cp/checkpoint-11750"
max_new_tokens:int  = 100
path_dataset: str = "../data/train_dataset.json"

In [ ]:
# source : https://colab.research.google.com/drive/1DqKNPOzyMUXmJiJFvJITOahVDxCrA-wA#scrollTo=9Ixtdtpgyv_a

from transformers import AutoModelForCausalLM, AutoTokenizer

# loads generative model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch_dtype)
tokenizer = AutoTokenizer.from_pretrained(model_name, torch_dtype=torch_dtype)
tokenizer.pad_token = tokenizer.eos_token # add a padding token, otherwise it raises an error

In [ ]:
from transformers.pipelines import pipeline
# loads pipeline to keep a view on not fine tuned model

raw_model_pipeline = pipeline("text-generation", model=model_name, tokenizer=tokenizer, max_new_tokens=max_new_tokens)

## 1 - Loads the training dataset in a hugging face Dataset

In [ ]:
import json
from datasets import Dataset
import random

with open(path_dataset, "rt") as f:
    train_dataset = json.load(f)

train_dataset = train_dataset[:int(data_prop*len(train_dataset))]
print(f"Length of dataset : {len(train_dataset)}")

tokenized_conversations = tokenizer.apply_chat_template(
    conversation=[each_acro["conversation"][0] for each_acro in train_dataset],
    return_tensors="pt",
    return_dict=True,
    truncation=True,
    padding=True,
    max_length=256,
)

tokenized_conversations["labels"] = tokenized_conversations["input_ids"]

conv_idx_for_test: int = random.randint(0, len(train_dataset)-1) # take one conversation for test
test_conv = train_dataset[conv_idx_for_test]

train_dataset = Dataset.from_dict(tokenized_conversations)

print(f"Example of conversation : {test_conv}")

In [ ]:
# view on dataset

train_dataset

## 2 - Training

In [ ]:
from training_tools import print_trainable_parameters, CustomCallback

def last_layers_fine_tuning(model):

    # trick to speed up training : freeze all layers except the last one
    for name, param in model.named_parameters():
        # print(f"{name}   Modelsize: {param.numel()/1000**2:.1f}M parameters")
        if "15" not in name:
            param.requires_grad = False
    return model

In [ ]:
from peft import LoraConfig, get_peft_model

def get_peft_config():
    return LoraConfig(
        r=16,
        lora_alpha=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_dropout=0.1,
        bias="lora_only",
        modules_to_save=["decode_head"]
    )

def lora_fine_tuning(model):
    config = get_peft_config()
    lora_model = get_peft_model(model, config)
    return lora_model

In [ ]:
from trl import SFTConfig, SFTTrainer

to_train_model = lora_fine_tuning(model) if do_lora else last_layers_fine_tuning(model)
print_trainable_parameters(to_train_model)



# Initialize trainer
training_args = SFTConfig(
    output_dir=output_dir,
    # max_steps=100,
    num_train_epochs=4 if data_prop < .1 else 2,
    per_device_train_batch_size=1,
    learning_rate=8e-5 if do_lora else 3e-5,
    logging_steps=50,
    # warmup_ratio=.0 if do_lora else .0,
    # save_steps=100,
    # eval_strategy="steps",
    # eval_steps=50,
)

trainer = SFTTrainer(
    model=to_train_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    peft_config=get_peft_config() if do_lora else None,
)

ft_model_pipeline = pipeline("text-generation", model=trainer.model, tokenizer=tokenizer, max_new_tokens=max_new_tokens)

trainer.add_callback(CustomCallback(raw_model_pipeline=raw_model_pipeline, ft_model_pipeline=ft_model_pipeline, test_conv=test_conv))

In [ ]:
trainer.train()

## Test the model

In [ ]:
model.eval() # eval mode : stops useless gradient computations

In [ ]:
def q_a(question, max_tokens: int = max_new_tokens):
    return ft_model_pipeline([{
        "role": "user",
        "content": question
    }], max_new_tokens=max_tokens)[0]["generated_text"][1]["content"]

In [ ]:
for i in range(10):
    print(q_a("What is Juropa ?")) 
    print("--------\n")

In [ ]:
q_a("What is Juropa in the field of cooking ?") # small check for overfitting

In [ ]:
q_a("What is Juropa ? In which country does it takes place ?", max_tokens=200) # test with new questions

In [ ]:
date = "06_05_25"
# trainer.save_model(f"../models/{date}") # saves the model